In [ ]:
data2=adata.to_df()

In [ ]:
from numpy.random import default_rng
rng = default_rng()

#create array of in cluster vs out of cluster gene data, use clustersizes to track which cluster bc 3d arrays are hard
clustersizes=[]
#for each cluster, find highest DEGs between that cluster and all the other clusters
for i in range(10):
    cluster = get_cluster(adata, np.asarray(data2), i)
    anticluster = get_anticluster(adata, np.asarray(data2), i)
    downsample = rng.choice(anticluster, size=cluster.shape[0], replace=False) #don't think this is necessary for ttest but included it for ease 
    
    clustersizes.append(cluster.shape[0])
    if i==0:
        clusters = cluster
        anticlusters = downsample
    else:
        clusters = np.append(clusters, cluster, axis=0)
        anticlusters = np.append(anticlusters, downsample, axis=0)

In [ ]:
clusters = np.transpose(np.asarray(clusters))
clusters.shape
anticlusters = np.transpose(np.asarray(anticlusters))
anticlusters.shape

In [ ]:
#for each cluster, compute a ttest for expression of genes in cluster vs out of cluster
import scipy
for n in range(len(clustersizes)):
    pval=[]
    start = int(np.sum(clustersizes[0:n]))
    end = int(np.sum(clustersizes[0:(n+1)]))
    for i in range(clusters.shape[0]):
        pval.append(scipy.stats.ttest_ind(clusters[i][start:end],anticlusters[i][start:end])[1])
    if n==0: pvals = [pval]
    else: pvals = np.append(pvals, [pval],axis=0)

In [ ]:
#take top 500 DEGs for each cluster and append to list to create full list of genes to do nmf on
for i in range(pvals.shape[0]):
    if i==0: 
        degind=[min_p(pvals[0],500)]
    else: 
        degind=np.append(degind, [min_p(pvals[i],500)],axis=0)

In [ ]:
allgenes=degind.flatten()

In [ ]:
#limit count matrix to DEGs (UNNORMALIZED)
data_np=np.asarray(data2)
reduced = []
reduced_genes = []
for i in range(data.shape[1]):
    if i in allgenes:
        reduced.append(data_np[:,i])
        reduced_genes.append(genes[i])
reduced=np.transpose(np.asarray(reduced))

In [ ]:
#THIS IS WHAT THE MODEL IS RUN ON
red=pd.DataFrame(reduced, columns=reduced_genes)

n_components = 10
H10,W10,W_norm10 = nmf_function(n_components,'nndsvd',1,red)

In [ ]:
def min_p(pvals,n):
    degind=[] #indices of most differentially expressed genes (lowest p values), in order

    minindex=-1
    #initialize minindex to an index that isn't nan
    for i in range(len(pvals)):
        if np.isnan(pvals[i])==False:
            minindex=i
            break
    if minindex==-1:
        return []
    
    #take top n DEGs
    for j in range(n):
        for i in range(len(pvals)):
            if (pvals[i]<pvals[minindex]):
                if (np.isnan(pvals[i])==False):
                    minindex=i
        degind.append(minindex)
        pvals[minindex]=100
    return degind

In [ ]:
def get_cluster(adata, factorized, n):
#only look at one cluster
    cluster1=[]
    for i in range(adata.obs["clusters"].shape[0]):
        if adata.obs["clusters"][i]==str(n):
            cluster1.append(factorized[i])
    return np.asarray(cluster1)

In [ ]:
def get_anticluster(adata, factorized, n):
#look at everything not in the cluster
    cluster1=[]
    for i in range(adata.obs["clusters"].shape[0]):
        if adata.obs["clusters"][i]!=str(n):
            cluster1.append(factorized[i])
    return np.asarray(cluster1)